##Data Prep

## 2.1 Gather and Explore the Dataset

This is for to show what is what !
not the actual code or model
Get a dataset with (article, summary) pairs.




# Dataset:

mounting google drive

In [ ]:
# mounting the drive
import os
from google.colab import drive

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')
    print("Drive is mounted.")
else:
    print("Drive is already mounted.")

Drive is already mounted.


- as seen in the output of this code block dataset include;

  only training split is dowloaded to drive.
  all training data's size.
  each data contains
      - {'aid': ..., 'mid': ..., 'ref_abstract': ..., 'abstract': ...}

  aid : article id of the actual paper.

  mid : reference id of cited or reference paper.

  ref_abstract : abstracts of multiple cited or reference papers. Input documents for the model.

  abstract : summary of the paper, ground truth for the model.

  article id and their abstract summary

In [ ]:
from datasets import load_dataset
import textwrap
!pip install datasets==2.0 --quiet
import textwrap
import pandas as pd


from datasets import load_dataset
save_dir = "/content/drive/MyDrive/00_Bireysel/02_dersler/nlp/project/my_paper/resources"
save_path = os.path.join(save_dir, "multi_science_dataset.json")

if not os.path.exists(save_path):
  print("Dataset downloading...")
  multi_xscience = load_dataset("bigbio/multi_xscience")
  multi_xscience['train'].to_json(save_path)
  multi_xscience = multi_xscience[multi_xscience["train"]]
  print("Dataset downloaded and saved")
else:
  multi_xscience_train = load_dataset("json", data_files=save_path, split="train")
  print("Dataset taken from drive ")


print(f'-'*200)
content_of_dataset = multi_xscience_train.features
print(f"content_of_dataset : {content_of_dataset}")
print(f'-'*200)

#for split in multi_xscience:
#    print(f"Split: {split}")
data = multi_xscience_train
num_samples = len(data)
print(f"Total samples: {num_samples}")

print(f'-'*200)
random_index = 0
example = multi_xscience_train[random_index]
article_id = example['aid']
print(f"article_id : {article_id }")
print(f'-'*200)

reference_paper = example["mid"]
print(f"reference_paper : {reference_paper}")
print(f'-'*200)

# looping through abstract references.
print("Reference Abstracts (input):")
for ref in example["ref_abstract"]["abstract"]:
  print("  - " + textwrap.fill(ref, width=200))
  print(f'-'*200)

print("Summary")
print("  - " + textwrap.fill(example["abstract"], width=200))


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-7bca471e353069d4/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.
Dataset taken from drive 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
content_of_dataset : {'aid': Value(dtype='string', id=None), 'mid': Value(dtype='string', id=None), 'abstract': Value(dtype='string', id=None), 'ref_abstract': {'mid': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'abstract': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Total samples: 30369
---------------

# some visualization

# Tokenization


- vocabulary
  combining all words from source data to create a vocabulary. lowercase all letters and splitting the words.
  using word2idx each word is mapped to unique ID.
- what is tokenization :
  
    converting raw text into tokens, after that each token is mapped to an id so that it can be used as input for models.

- what are the parameters
      max_legth
    is the maximum number of tokens in encoded sentence.
      <pad>
    in case of shorter sentence padding ensures uniform length
      <unk>
    representing words that are  not on the vocablary.
      truncate
    cutting the sentence if it exceeds the max length.

- input for tokenization and output of tokenization :

  normal source text becomes a list of tokens which include their unique id.
  If these tokens are decoded, original words can be extracted with paddings(< pad >)



In [ ]:
from collections import Counter

class SimpleTokenizer:
    full_size = 269401
    def __init__(self, data, vocab_limit=25000):  #
                                        #269401
        self.data = data
        self.vocab_limit = vocab_limit
        self.build_vocab()
        print(f"vocab size : {vocab_limit}]")


    def build_vocab(self):
        word_counter = Counter()
        for item in self.data:
            for text in item["ref_abstract"]["abstract"]:
                word_counter.update(text.lower().split())
            word_counter.update(item["abstract"].lower().split())

        most_common_words = word_counter.most_common(self.vocab_limit)
        self.word2idx = {"<pad>": 0, "<unk>": 1}
        for idx, (word, _) in enumerate(most_common_words, start=2):
            self.word2idx[word] = idx

        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

    def encode(self, text, max_length):
        tokens = text.lower().split()
        token_ids = [
            self.word2idx.get(word, self.word2idx["<unk>"]) for word in tokens
        ]
        if len(token_ids) < max_length:
            token_ids += [self.word2idx["<pad>"]] * (max_length - len(token_ids))
        else:
            token_ids = token_ids[:max_length]
        return token_ids

    def decode(self, token_ids):
        return " ".join(self.idx2word.get(idx, "<unk>") for idx in token_ids)

    def get_vocab_size(self):
        return len(self.word2idx)


## dummy showcase

using the dummy dataset we can observe the input and output of this process

In [ ]:
dummy_train_data = [multi_xscience_train[0]]  # Only 1 sample

source_text = " ".join(dummy_train_data[0]["ref_abstract"]["abstract"])
print(textwrap.fill(source_text, width=200))
print("-" * 200)

simple_tokenizer = SimpleTokenizer(dummy_train_data)
encoded = simple_tokenizer.encode(source_text, max_length = 153)
print(f"encoded : {encoded}")
print("-" * 200)

decoded = simple_tokenizer.decode(encoded)
print(textwrap.fill(decoded, width=200))

This note is a sequel to our earlier paper of the same title [4] and describes invariants of rational homology 3-spheres associated to acyclic orthogonal local systems. Our work is in the spirit of
the Axelrod–Singer papers [1], generalizes some of their results, and furnishes a new setting for the purely topological implications of their work. Recently, Mullins calculated the Casson-Walker
invariant of the 2-fold cyclic branched cover of an oriented link in S^3 in terms of its Jones polynomial and its signature, under the assumption that the 2-fold branched cover is a rational homology
3-sphere. Using elementary principles, we provide a similar calculation for the general case. In addition, we calculate the LMO invariant of the p-fold branched cover of twisted knots in S^3 in terms
of the Kontsevich integral of the knot.
----------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Encoder

- Encoders :

  encoders takes sequence of tokens and turn them into hidden representaion. In this case using embedding layer and Long Short-Term Memory (LSTM) layer.

- Parameters :
   
      vocab_size  
  is obtained from the tokenizer class
      embed_dim
  how many numbers represent each wordç
      hidden_dim
  size of LSTM's memory
      num_layers
  number of stacked LSTM layers
    


- pipeline itself
      
  1 - embedding_layer
  
  input : token id shape is (1, 153(max_legth))
      embed_dim = 128
  output : embedded vectors with shape (1, 153, 128)
  1 : number of sequence.
  153 : number of tokens
  128 : each word become dimensional vector.

  2 - LSTM Layer

  input : (1, 153, 128)
  for each word LSTM outpur (1, 153, 256) 256-dim vector
  with two LSTM layers (2, 153, 256)




  Output


      outputs
  LSTM output at each time sterp.

  embeddings are located here.

      hidden
  final hidden state of each LSTM layer. (initializing decoder)
    a kind of memory of LSTM
      cell
  
  internal memory of each LSTM(initializing decoder)


In [ ]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
    print(f"self.embedding : {self.embedding}") # DB - Size of your vocab (136 unique tokens: words + <pad> + <unk>)
    print(f"self.lstm : {self.lstm}")           # DB - Each word will be represented as a vector of 128 numbers (dense, not one-hot).
  def forward(self, input_ids):
    embedded = self.embedding(input_ids)               # (batch_size, seq_len, embed_dim)
    outputs, (hidden, cell) = self.lstm(embedded)       # (batch_size, seq_len, hidden_dim)
    return outputs, hidden, cell

In [ ]:
## DUMMY USECASE !
import torch
import textwrap

dummy_train_data = [multi_xscience_train[0]]

# creating a dummy dataset
source_text = " ".join(dummy_train_data[0]["ref_abstract"]["abstract"])
print(textwrap.fill(source_text, width=200))
print("-" * 200)

# using tokenizer
simple_tokenizer = SimpleTokenizer(dummy_train_data)

encoded = simple_tokenizer.encode(source_text, max_length=153)

decoded = simple_tokenizer.decode(encoded)
input_tensor = torch.tensor([encoded], dtype=torch.long)  # batch_size = 1

# Initializing Encoder
vocab_size = simple_tokenizer.get_vocab_size()
print(f"vocab_size : {vocab_size}")
embed_dim = 128
hidden_dim = 256
num_layers = 2

encoder = Encoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers
)

# pass through encoder !
outputs, hidden, cell = encoder(input_tensor)

# 9. Print what went into encoder and what came out
print(f"Input tensor shape (to Encoder) : {input_tensor.shape}")     # (batch_size, seq_len)
print(f"Outputs shape (from Encoder)    : {outputs.shape}")          # (batch_size, seq_len, hidden_dim)
print(f"Hidden state shape              : {hidden.shape}")           # (num_layers, batch_size, hidden_dim)
print(f"Cell state shape                : {cell.shape}")             # (num_layers, batch_size, hidden_dim)
print("-" * 200)
print(outputs[0, :5, :10])  # first 5 tokens, first 10 hidden dimensions
print(hidden[:, 0, :10])    # hidden states (all layers), first 10 hidden dims
print(cell[:, 0, :10])      # cell states (all layers), first 10 hidden dims

# Word "the" → [0.12, -0.44, 0.78, ..., 0.01] (a vector of 128 numbers)

This note is a sequel to our earlier paper of the same title [4] and describes invariants of rational homology 3-spheres associated to acyclic orthogonal local systems. Our work is in the spirit of
the Axelrod–Singer papers [1], generalizes some of their results, and furnishes a new setting for the purely topological implications of their work. Recently, Mullins calculated the Casson-Walker
invariant of the 2-fold cyclic branched cover of an oriented link in S^3 in terms of its Jones polynomial and its signature, under the assumption that the 2-fold branched cover is a rational homology
3-sphere. Using elementary principles, we provide a similar calculation for the general case. In addition, we calculate the LMO invariant of the p-fold branched cover of twisted knots in S^3 in terms
of the Kontsevich integral of the knot.
----------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Decoder

In terms of structure decoder and encoder are similar.

both use embedding layyers to convert tokens to vectors

use LSTM to process sequence of data

start with <sos> token then continue one by one.
It outputs a sequence of token prediction.

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
    self.fc_out = nn.Linear(hidden_dim, vocab_size)

  def forward(self, input_ids, hidden, cell):
    embedded = self.embedding(input_ids)  # (batch_size, seq_len, embed_dim)
    outputs, (hidden, cell) = self.lstm(embedded, (hidden, cell))  # Use the previous hidden and cell states
    predictions = self.fc_out(outputs)  # (batch_size, seq_len, vocab_size)
    return predictions, hidden, cell

# seq2seq

##

encoder converts input to memory

decoder create new output from memory(encoder)


seq2seq handles one full forward pass!

      def forward(self, src, trg)

  forward is the actual method, responsible for the pass through network.
  
      src
  represent the input ensor with shape of batch size and src_seq_len

      trg
  ground truth output



##flow

output of forward method's one pass  



In [ ]:
# actual seq2seq class !
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg):
    batch_size = src.shape[0]
    trg_len = trg.shape[1]
    vocab_size = self.decoder.fc_out.out_features

    outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)

    encoder_outputs, hidden, cell = self.encoder(src)

    input_token = trg[:, 0].unsqueeze(1)
    for t in range(1, trg_len):
        output, hidden, cell = self.decoder(input_token, hidden, cell)
        outputs[:, t] = output.squeeze(1)

        top_tok = output.argmax(2)
        input_token = top_tok

    return outputs


# Dataset Class

actual dataset class to better use our data

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
  def __init__(self, data, tokenizer, max_input_len=256, max_target_len=128):
    self.data = data
    self.tokenizer = tokenizer
    self.max_input_len = max_input_len
    self.max_target_len = max_target_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    item = self.data[idx]

    # Prepare source and target texts
    source_text = " ".join(item["ref_abstract"]["abstract"])
    target_text = item["abstract"]

    # Tokenize
    source_ids = self.tokenizer.encode(source_text, self.max_input_len)
    target_ids = self.tokenizer.encode(target_text, self.max_target_len)

    return {
      "input_ids": torch.tensor(source_ids, dtype=torch.long),
      "labels": torch.tensor(target_ids, dtype=torch.long)
    }


# initilaizing all


In [ ]:
from torch.utils.data import DataLoader

train_dataset = MyDataset(multi_xscience_train, simple_tokenizer)
# Select subset
print(f"data set ength : {len(train_dataset)}")
reduced_data = multi_xscience_train.select(range(len(train_dataset)))

# Create tokenizer
simple_tokenizer = SimpleTokenizer(reduced_data)
#simple_tokenizer = SimpleTokenizer(reduced_data, vocab_limit=10000)

# Wrap with dataset
train_dataset = MyDataset(reduced_data, simple_tokenizer)

# Create DataLoader
# 128/512/256
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers = 16, pin_memory = True)


data set ength : 30369
vocab size : 25000]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# training loop

      device

  if available choose a CPU cuda else runs on CPU

  These are hyperparameters that effect model's performance.

      vocab_size
      embed_dim = 256

  - it is how many numbers represent each word(token)
  - higher dimension meaning a richer representation with more memory usage.
  - much higher dimensions is not neccesary in small datasets.


      hidden_dim = 512

  
  - size of the hidden state in LSTM.
  - it is responsible for how much info LSTM can store and process.
  - bigger memory meaning better "thinking", yet can be expensive computationaly.
  - higher risk of overfitting in small datasets.     
      

      num_layers = 2

- number of stacked LSTM layers.
- more layer a better model that can comprehend more complex patterns.
- too many layers create diffuculty in training.
  
    

  setting up values for encoder, decoder and wrap them into seq2seq model.

      encoder = Encoder(vocab_size, embed_dim, hidden_dim, num_layers)
      decoder = Decoder(vocab_size, embed_dim, hidden_dim, num_layers)
      model = Seq2Seq(encoder, decoder, device).to(device)
----------------------------------------------------

based on how our model learns we can adjust these values.

      optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

optimizer :
  
  - controls how the model updates its weights, according to the loss.

ADAM - Adaptive Moment Estimation :
  - chosen according to popularity amoung ML applications.

  - using momentum it updates learning rates adaptively.

learning rate :
  - chosen according to safe value for ADAM optimizer.  
  - important hyperparameter.
  - controls how big each update is to the weights.
   
   *small learning rate = > model learns slow
   
   *big model skip important details.
- - - - - -- - - - - - -


loss function :

- telling the model, how wrong it is. Since we are predicting tokens from vocablary CrossEntropyLoss is good choice.

- <pad> tokens are ignored, since they represent padding, not actually meaningfull words.

      criterion = nn.CrossEntropyLoss(ignore_index=simple_tokenizer.word2idx["<pad>"])



Epochs :

this is where the actual loop is located.
1 epoch meaning a full pass through the dataset.
      
      num_of_epochs

decide how many times dataset will go through the model during training.

-----------------------------------


      for i, batch in enumerate(train_loader):
        src = batch["input_ids"].to(device)
        trg = batch["labels"].to(device)
  
iterating in training data in mini-batches.




      input = batch["input_ids"].to(device)
      target = batch["labels"].to(device)


- src : input, target : target summary as tensors.
- these tensors are moved to device(GPU or CPU)

      optimizer.zero_grad()

- clears ol gradients from last step. precent gradient accumulation.


      
      output = model(inputs, target)
- passes the source and target througgh seq2seq model
output : is a tensor of predicted word probabilities, also called logits

      output_dim = output.shape[-1]    
      output = output[:, 1:].reshape(-1, output_dim)
      target = target[:, 1:].reshape(-1)

- skipping the sos token
- flattening predictions and targets for CrossEntropyLoss

output - shape => [batch_size * seq_len, vocab_size]

target - shape => [batch_size * seq_len]

--------------------------------

      loss = criterion(output, target)

- comparing model prediction and actual token
- how wrong the prediction is
------------------------

      loss.backward()
- backpropagation step, calculate gradients for all parameters and observe how much each is contributed to the loss.
- how much we need to adjust each weight for reduced loss.
----------------------------------

      optimizer.step()
- updating the model weights according yo previous calculated(loss.backward).

- optimizer uses gradients and learning rate to adjust weights.

--------------

Every 10 batches, the batch number is displayed.
the average loss is calculated to observe training performance.

# training code

In [ ]:
# Step 5 - Training
train = False
if train:

  device = torch.device("cuda")

  vocab_size = simple_tokenizer.get_vocab_size()
  embed_dim = 256
  hidden_dim = 512
  num_layers = 2

  encoder = Encoder(vocab_size, embed_dim, hidden_dim, num_layers)
  decoder = Decoder(vocab_size, embed_dim, hidden_dim, num_layers)
  model = Seq2Seq(encoder, decoder, device).to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss(ignore_index=simple_tokenizer.word2idx["<pad>"])

  num_of_epochs = 15

  for epoch in range(num_of_epochs):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(train_loader):
      inputs = batch["input_ids"].to(device)
      target = batch["labels"].to(device)

      optimizer.zero_grad()

      output = model(inputs, target)

      output_dim = output.shape[-1]
      output = output[:, 1:].reshape(-1, output_dim)
      target = target[:, 1:].reshape(-1)

      loss = criterion(output, target)

      loss.backward()
      optimizer.step()
      if i % 10 == 0:
        print(f"  Batch {i}/{len(train_loader)}")

      epoch_loss += loss.item()


    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_of_epochs}], Loss: {avg_loss:.4f}")



  import os
  number_of_revision = 1
  save_path = "/content/drive/MyDrive/00_Bireysel/02_dersler/nlp/project/my_paper"
  os.makedirs(save_path, exist_ok=True)
  save_name = f"{number_of_revision}_seq2seq_model_5.pth"
  torch.save(model.state_dict(), os.path.join(save_path, save_name))

  import json
  with open(os.path.join(save_path, "vocab.json"), "w") as f:
      json.dump(simple_tokenizer.word2idx, f)

  print(f"Model and tokenizer saved to {save_path}")
else:
  pass

In [ ]:
import json
with open("/content/drive/MyDrive/00_Bireysel/02_dersler/nlp/project/my_paper/vocab.json", "r") as f:
    word2idx = json.load(f)

vocab_size = simple_tokenizer.get_vocab_size()
# These parameters need to change according to model itself.
print(f"vocab_size : {vocab_size}")
embed_dim = 256    #
hidden_dim = 128   #
num_layers = 2
device = torch.device("cpu")

encoder = Encoder(vocab_size, embed_dim, hidden_dim, num_layers)
decoder = Decoder(vocab_size, embed_dim, hidden_dim, num_layers)
model = Seq2Seq(encoder, decoder, device)

# Loading the model
# using cpu just to predict.
model.load_state_dict(torch.load(
    "/content/drive/MyDrive/00_Bireysel/02_dersler/nlp/project/my_paper/best_model.pth",
    map_location="cpu"
))
model.eval()

TypeError: SimpleTokenizer.get_vocab_size() missing 1 required positional argument: 'self'

In [ ]:
def predict(model, tokenizer, input_text, max_len=64):
  model.eval()
  device = torch.device("cpu")

  input_ids = tokenizer.encode(input_text, max_length=128)
  input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)


  with torch.no_grad():
    _, hidden, cell = model.encoder(input_tensor)

  decoder_input = torch.tensor([[tokenizer.word2idx.get("<pad>", 0)]], dtype=torch.long).to(device)
  outputs = []

  for _ in range(max_len):
    with torch.no_grad():
      output, hidden, cell = model.decoder(decoder_input, hidden, cell)
      token = output.argmax(2)
      token_id = token.item()
      outputs.append(token_id)

    decoder_input = token

    if token_id == tokenizer.word2idx.get("<pad>", 0):
      break

  return tokenizer.decode(outputs)


In [ ]:
input_text = (
    "They are able to transform one sequence to another, meaning it can understand the context of the text and act accordingly. "

)

summary = predict(model, simple_tokenizer, input_text)
print("Summary:\n", summary)


In [ ]:
for i, (word, idx) in enumerate(word2idx.items()):
    print(f"{i}: {word} -> {idx}")
    if i > 100: break
